In [72]:
import pandas as pd
import numpy as np
from numpy import random
import math

# random.seed(100)

* TicketID
* Datetime
* Type : ["Security", "Account", "Database", "Application", "Network"]
* Priority
* RequestingDepartment : ["Sales", "Operators", "Production", "Reporting", "Receiving", "Warehouse", "Marketing"]

In [66]:
# Create a df of companies

def create_companies(cnt_companies=50):
    Company_df = pd.DataFrame({
        "CompanyID": pd.Series(dtype=str), "TotalEmployees": pd.Series(dtype="str"), "CompanyWeight": pd.Series(dtype=float), "CompanyWeightRange": pd.Series()  
    })

    employee_range = ["10-50", "10-50", "10-50", "10-50", "10-50", "10-50", "10-50", "10-50", 
                    "50-100", "50-100", "50-100", "50-100", 
                    "100-500", "100-500", "100-500", 
                    "500-1,000", "500-1,000", 
                    "1,000-10,000", 
                    "10,000+", "10,000+"]
    company_weight = {
        "10-50": [0.1, 0.5], "50-100": [0.5, 1.0], "100-500": [1.0, 2.0], "500-1,000": [2.0, 4.0], "1,000-10,000": [4.0, 6.0], "10,000+": [6.0, 10.0],
    }
    
    Company_df["CompanyID"] = "Co." + random.randint(1000, 9999, cnt_companies).astype(str)
    Company_df["TotalEmployees"] = random.choice(employee_range, cnt_companies)
    Company_df["CompanyWeightRange"] = Company_df["TotalEmployees"].map(company_weight)

    for i in range(len(Company_df)):
        Company_df.iloc[i, 2] = random.uniform(Company_df["CompanyWeightRange"][i][0], Company_df["CompanyWeightRange"][i][1], 1)

    return Company_df

company_df = create_companies()

In [92]:
date_df = pd.DataFrame({"Date": pd.date_range(start="2026-01-01", end="2026-01-31").tolist()})
hours_df = pd.DataFrame({"Hours": np.arange(0, 24, 1)})

date_df = date_df.join(hours_df, how="cross")

iter_df = pd.DataFrame({"SubmitTime": date_df["Date"] + pd.to_timedelta(date_df["Hours"], unit="h")})
df = pd.DataFrame(columns=["TicketID", "SubmitTime", "CloseTime", "DurationMinutes", "Type", "Priority", "CompanyID"])

types = ["Security", "Account", "Database", "Application", "Network", "Login"]

for i in range(len(iter_df)):
    SubmitTime = iter_df["SubmitTime"].iloc[i]

    for j in range(len(company_df)):
        company_id = company_df.iloc[j, 0]
        company_weight = company_df.iloc[j, 2]
        # converted_weight = 0 
        # if company_weight < 1:
        #     converted_weight = 1
        # else: 
        #     converted_weight = math.ceil(company_weight)

        cnt_tickets = random.randint(0, (1 * math.ceil(company_weight)) + 1) # Tickets per hour
        join_df = pd.DataFrame()

        join_df["TicketID"] = random.randint(10000, 99999, size=cnt_tickets)
        join_df["SubmitTime"] = pd.to_datetime(SubmitTime)  # Single timestamp, broadcasted
        join_df["DurationMinutes"] = pd.to_timedelta(np.random.randint(0, 121, size=cnt_tickets), unit='m')
        join_df["CloseTime"] = join_df["SubmitTime"] + join_df["DurationMinutes"]   
        join_df["Type"] = random.choice(types, size=cnt_tickets)
        join_df["Priority"] = random.randint(0, 2, cnt_tickets)
        join_df["CompanyID"] = [company_id for i in range(cnt_tickets)]

        join_df["DurationMinutes"] = (join_df["DurationMinutes"] / pd.Timedelta(minutes=1)).astype(int)   

        df = pd.concat([df, join_df])

df

,TicketID,SubmitTime,CloseTime,DurationMinutes,Type,Priority,CompanyID
0,48921,2026-01-01 00:00:00,2026-01-01 00:48:00,48,Database,1,Co.3515
1,28275,2026-01-01 00:00:00,2026-01-01 01:31:00,91,Login,1,Co.3515
2,64804,2026-01-01 00:00:00,2026-01-01 01:24:00,84,Application,1,Co.3515
3,96244,2026-01-01 00:00:00,2026-01-01 01:46:00,106,Application,0,Co.3515
4,93993,2026-01-01 00:00:00,2026-01-01 00:19:00,19,Application,1,Co.3515
...,...,...,...,...,...,...,...
0,91985,2026-01-31 23:00:00,2026-01-31 23:59:00,59,Application,0,Co.4439
1,88021,2026-01-31 23:00:00,2026-02-01 00:07:00,67,Login,1,Co.4439
2,88739,2026-01-31 23:00:00,2026-02-01 00:23:00,83,Database,1,Co.4439
3,29704,2026-01-31 23:00:00,2026-01-31 23:31:00,31,Application,0,Co.4439


In [93]:
df = pd.merge(df, company_df, on="CompanyID", suffixes=("", "_y"))
df = df.drop("CompanyWeightRange", axis=1)

df.head(10)

,TicketID,SubmitTime,CloseTime,DurationMinutes,Type,Priority,CompanyID,TotalEmployees,CompanyWeight
0,48921,2026-01-01 00:00:00,2026-01-01 00:48:00,48,Database,1,Co.3515,"10,000+",7.060198
1,28275,2026-01-01 00:00:00,2026-01-01 01:31:00,91,Login,1,Co.3515,"10,000+",7.060198
2,64804,2026-01-01 00:00:00,2026-01-01 01:24:00,84,Application,1,Co.3515,"10,000+",7.060198
3,96244,2026-01-01 00:00:00,2026-01-01 01:46:00,106,Application,0,Co.3515,"10,000+",7.060198
4,93993,2026-01-01 00:00:00,2026-01-01 00:19:00,19,Application,1,Co.3515,"10,000+",7.060198
5,70081,2026-01-01 00:00:00,2026-01-01 01:03:00,63,Security,1,Co.3515,"10,000+",7.060198
6,39633,2026-01-01 00:00:00,2026-01-01 01:48:00,108,Database,0,Co.3515,"10,000+",7.060198
7,50089,2026-01-01 00:00:00,2026-01-01 01:17:00,77,Account,0,Co.5473,"500-1,000",3.161339
8,65331,2026-01-01 00:00:00,2026-01-01 00:36:00,36,Application,0,Co.5473,"500-1,000",3.161339
9,49283,2026-01-01 00:00:00,2026-01-01 00:07:00,7,Login,0,Co.5473,"500-1,000",3.161339


In [95]:
df.to_excel("IT_Ticket_Data.xlsx")